# Análisis estadístico básico de los grupos
En este notebook, se verá un análisis general para los grupos creados, esto con la intención de entender a grandes rasgos la composición de cada grupo

In [1]:
import sys
sys.path.insert(0, './scripts/')

In [2]:
from collections import defaultdict
import gc
import IDTMex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st


In [3]:
gc.collect()

0

In [4]:
pd.set_option('display.max_columns', None)

### Lectura de Datos

In [5]:
base_path = "./ConjuntosDatos/"
endutih_path = base_path + "ENDUTIH2020/conjuntos_de_datos/"

In [6]:
endutih_vivhogar = pd.read_csv(endutih_path+"viv_hog_agrupados.csv")
endutih_res = pd.read_csv(endutih_path+"residente_agrupados.csv")
endutih_usr = pd.read_csv(endutih_path+"usuario_agrupados.csv", low_memory=False)
endutih_usr2 = pd.read_csv(endutih_path+"usuario2_agrupados.csv", low_memory=False)
endutih_completa = pd.merge(endutih_vivhogar,endutih_res.drop(["UPM_DIS","ESTRATO","ENT","DOMINIO", "EST_DIS", "TLOC","Grupo"], axis=1), on = ["UPM","VIV_SEL", "HOGAR"])

In [7]:
estados = pd.read_csv(base_path+"CodigoEdos.csv")
estados

,Codigo,Nombre,ENT
0,1,Aguascalientes,Ags.
1,2,Baja California,B.C.
2,3,Baja California Sur,B.C.S
3,4,Campeche,Camp.
4,5,Coahuila de Zaragoza,Coah.
5,6,Colima,Col.
6,7,Chiapas,Chis.
7,8,Chihuahua,Chih.
8,9,Ciudad de México,C.D.Mx
9,10,Durango,Dgo.


# Todos los grupos

## Combinaciones

In [8]:
dist_all = endutih_vivhogar[["ENT","DOMINIO","Grupo", "FAC_HOG"]].groupby(["ENT","DOMINIO","Grupo"]).agg("sum").reset_index().rename(columns={"FAC_HOG":"Total"})
dist_all["ENT"] = dist_all.ENT.replace({codigo:nombre for codigo,nombre in zip(estados["Codigo"], estados["ENT"])},regex=True)
dist_all["DOMINIO"] = dist_all.DOMINIO.replace({"R":"Rural", "U":"Urbano"},regex=True)
dist_all["perc"] = (dist_all.Total/ dist_all.Total.sum()*100)
dist_all

,ENT,DOMINIO,Grupo,Total,perc
0,Ags.,Rural,20,86605,0.242699
1,Ags.,Rural,21,507,0.001421
2,Ags.,Rural,22,473,0.001326
3,Ags.,Rural,23,272,0.000762
4,Ags.,Urbano,20,113614,0.318389
...,...,...,...,...,...
515,Zac.,Urbano,32,149,0.000418
516,Zac.,Urbano,33,80273,0.224955
517,Zac.,Urbano,34,384,0.001076
518,Zac.,Urbano,42,31107,0.087173


## Distribución

In [10]:
dist_grupos = endutih_vivhogar[["ESTRATO","Grupo", "FAC_HOG"]].groupby(["ESTRATO","Grupo"]).agg("sum").reset_index()
dist_grupos

,ESTRATO,Grupo,FAC_HOG
0,1,11,6984839
1,2,20,17295201
2,2,21,108498
3,2,22,110877
4,2,23,92468
5,3,30,138234
6,3,31,81353
7,3,32,104798
8,3,33,7106718
9,3,34,43151


In [11]:
dist_grupos.Grupo.nunique()

15

Relativos al total de hogares

In [9]:
(dist_grupos/ dist_grupos.FAC_HOG.sum()*100)

FAC_HOG
ESTRATO Grupo           
1       11     19.574120
2       20     48.467593
        21      0.304052
        22      0.310719
        23      0.259130
3       30      0.387383
        31      0.227981
        32      0.293683
        33     19.915670
        34      0.120925
4       40      0.219829
        41      0.131499
        42      9.443502
        43      0.299554
        44      0.044362

## Concentración en estados

In [10]:
grupo_estados = pd.pivot_table(data=endutih_vivhogar, columns="ENT", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_estados.columns = estados.ENT
grupo_estados

ENT              Ags.    B.C.   B.C.S   Camp.   Coah.    Col.   Chis.   Chih.  \
ESTRATO Grupo                                                                   
1       11          0   52722   40490   94288   36088    6864  829585   85504   
2       20     200219  507326  111115  116090  400247  137829  464702  478467   
        21       1519    4220     846    1099    1979     572    2155       0   
        22       2593    2880    1160    1355    1994    1319    3528     626   
        23       1420    1709     684    1789    3652    1373    2577       0   
3       30       2227    7336    2127     513    6509     393     785    1650   
        31       2643    3634     674     515    1848     673    2034    1705   
        32        769    5485     105     245     537     212       0    1053   
        33     122136  334985   54272   21061  347796   35622   55827  398314   
        34        909    1279     680     399    1891     335       0     525   
4       40       1304    4844    1187     487    1238    1761       0     912   
        41       1413     670     394     156     550     582     677     549   
        42      45140  181461   34926   17174   98480   41080   20260  160656   
        43       1660    1246     102     145    2668     141     681     840   
        44        822    1462     139     116       0     283       0       0   

ENT             C.D.Mx    Dgo.    Gto.    Gro.    Hgo.     Jal.  Edo. Méx.  \
ESTRATO Grupo                                                                
1       11        4040   67991  244784  571453  221026    90597     589821   
2       20     1089994  284133  918089  323185  537452  1146740    2608220   
        21       11364     265    7161    1917    2819     7753      28016   
        22        3704     670   12024    2474    2361     5509      13199   
        23         908     238    9479    1125    2225     5185       2385   
3       30       32531       0    9714     454       0    16543      14126   
        31       12437     802    3978    1180     409     7568       9700   
        32       41001       0    4820       0     853    13531       4978   
        33     1193378  101901  274208   53578   41641   733640     719961   
        34        6117     203    5216       0       0     9146          0   
4       40       14164    1111    5134       0       0     6530      11313   
        41        6802     842    1038       0       0    11843          0   
        42      462092   47411  103019    1848   38859   274829     587679   
        43       28215       0    6096       0       0    15569      22043   
        44        1971       0    3003       0       0        0          0   

ENT             Mich.    Mor.    Nay.    N.L.    Oax.    Pue.    Qro.  \
ESTRATO Grupo                                                           
1       11     294591   68508   37842  137973  679375  575579  260256   
2       20     811481  371788  249027  625256  414561  816830  225819   
        21       6156    2935     834     926    1151     843    2803   
        22       5496    3714     767    3429    2029    5337    2703   
        23      18619    2069    1075       0    2006       0    1593   
3       30       2586    2686     809    9802    1842       0    1273   
        31       3511    1685     730    5068       0    1083    1458   
        32       1401    2179     302    5453     461    3306    2321   
        33     181981   73897   64788  516119   40322  225656   84143   
        34       2187     825     326     752     518       0     680   
4       40       1996       0     466   11848       0    1086    1299   
        41       1383     517       0    2597       0    2212     686   
        42      35488   22677   17266  321829    6538  105479   74424   
        43          0    2090     440   10032       0    2835    1408   
        44          0     108     179       0       0       0    1588   

ENT            Q. Roo.   S.L.P    Sin.    Son.    Tab

### Relativo al grupo

In [11]:
grupo_estados_rel_grupo = grupo_estados.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_estados_rel_grupo.style.format(precision=4)

De forma general, parece que hay una buena distribución de los grupos en los estados

### Relativo a la Entidad

In [12]:
grupo_estados_rel_ent = grupo_estados.apply(lambda x: x.div(x.sum()), axis=0)*100
grupo_estados_rel_ent.style.format(precision=4)

In [13]:
np.amax(grupo_estados_rel_ent)

C:\Users\jazmi\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


ENT
Ags.         52.035481
B.C.         45.653264
B.C.S        44.642247
Camp.        45.448495
Coah.        44.202890
Col.         60.177088
Chis.        59.992653
Chih.        42.312219
C.D.Mx       41.027628
Dgo.         56.200860
Gto.         57.103503
Gro.         59.699607
Hgo.         63.405317
Jal.         48.901847
Edo. Méx.    56.559761
Mich.        59.367565
Mor.         66.907094
Nay.         66.433596
N.L.         37.869424
Oax.         59.137642
Pue.         46.937617
Qro.         39.286652
Q. Roo.      57.382214
S.L.P        35.339037
Sin.         50.241358
Son.         43.645319
Tab.         57.578661
Tamps.       54.322547
Tlax.        81.637524
Ver.         50.260302
Yuc.         42.175304
Zac.         59.793218
dtype: float64

La mayoría se encuentra en el grupo 20, que es donde está casi la mitad de los hogares mexicanos

## Internet

In [14]:
grupo_internet = pd.pivot_table(data=endutih_vivhogar, columns="P4_4", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_internet.columns = pd.Series(["Sí", "No"], name="Internet en el hogar")
grupo_internet

Internet en el hogar        Sí       No
ESTRATO Grupo                          
1       11             1767283  5217556
2       20            10168088  7127113
        21              108134      364
        22              110877        0
        23               47795    44673
3       30              125409    12825
        31               59912    21441
        32              104798        0
        33             5561698  1545020
        34               43151        0
4       40               75734     2710
        41               37829     9095
        42             3055407   314417
        43              106893        0
        44               15830        0

### Relativo al grupo

In [15]:
grupo_internet_rel_grupo = grupo_internet.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_internet_rel_grupo.style.format(precision=4)

Hay variaciones dentro de los grupos, sin embargo, destaca el hecho de que hay grupos donde todos los hogares tienen Internet

### Relativo a la Disponibilidad de internet

In [16]:
grupo_internet_rel_int = grupo_internet.apply(lambda x: x.div(x.sum()), axis=0)*100
grupo_internet_rel_int.style.format(precision=4)

### Internet Detalles

In [17]:
grupo_internet_tipo = pd.pivot_table(data=endutih_vivhogar.loc[endutih_vivhogar.P4_4==1], columns="P4_5", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_internet_tipo.columns = pd.Series(["Fija", "Móvil", "Ambas", "No sabe"], name="Tipo de Internet en el hogar")
grupo_internet_tipo

Tipo de Internet en el hogar     Fija   Móvil    Ambas  No sabe
ESTRATO Grupo                                                  
1       11                    1129636  415595   214295     7757
2       20                    6785857  768901  2576932    36398
        21                      76096     750    27372     3916
        22                      84507       0    23244     3126
        23                      36662    3387     5906     1840
3       30                      78400    4079    42930        0
        31                      42975     452    14191     2294
        32                      56224       0    48574        0
        33                    3290073  187424  2076314     7887
        34                      26185       0    15539     1427
4       40                      40709     221    34804        0
        41                      17155     156    20518        0
        42                    1831189   54078  1160692     9448
        43                      48549    1740    56604        0
        44                       9351       0     6479        0

#### Relativo al grupo

In [18]:
grupo_internet_tipo_rel_grupo = grupo_internet_tipo.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_internet_tipo_rel_grupo.style.format(precision=4)

De los hogares que tienen internet, en su mayoría lo tienen de forma fija

#### Relativo al tipo de Internet

In [19]:
grupo_internet_tipo_rel_tipo = grupo_internet_tipo.apply(lambda x: x.div(x.sum()), axis=0)*100
grupo_internet_tipo_rel_tipo.style.format(precision=4)

## Electricidad

In [20]:
grupo_electricidad = pd.pivot_table(data=endutih_vivhogar, columns="P1_4", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_electricidad.columns = pd.Series(["Sí", "No"], name="Electricidad")
grupo_electricidad

Electricidad         Sí      No
ESTRATO Grupo                  
1       11      6870009  114830
2       20     17249295   45906
        21       108498       0
        22       110877       0
        23        92468       0
3       30       138234       0
        31        81353       0
        32       104798       0
        33      7092239   14479
        34        43151       0
4       40        78444       0
        41        46924       0
        42      3368593    1231
        43       106893       0
        44        15830       0

### Relativo al grupo

In [21]:
grupo_electricidad_rel_grupo = grupo_electricidad.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_electricidad_rel_grupo.style.format(precision=4)

Como la mayoría de los hogares tienen electricidad, no consideraremos este factor como una limitante

## Computadora

In [22]:
endutih_vivhogar["COMPU"] = (endutih_vivhogar[["P4_2_1", "P4_2_2", "P4_2_3"]]==1).any(axis=1)

In [23]:
grupo_compu = pd.pivot_table(data=endutih_vivhogar, columns="COMPU", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_compu.columns = pd.Series(["No", "Sí"], name="Computadora en el hogar")
grupo_compu

Computadora en el hogar        No       Sí
ESTRATO Grupo                             
1       11                5874989  1109850
2       20               10410499  6884702
        21                  28615    79883
        22                  48555    62322
        23                  54205    38263
3       30                  41005    97229
        31                  36945    44408
        32                  22979    81819
        33                2825919  4280799
        34                  16688    26463
4       40                  16268    62176
        41                  13491    33433
        42                 666910  2702914
        43                   7685    99208
        44                   4009    11821

### Relativo al grupo

In [24]:
grupo_compu_rel_grupo = grupo_compu.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_compu_rel_grupo.style.format(precision=4)

Vemos que en su mayoría no cuentan con un equipo de cómputo

### Detalles

In [25]:
grupo_compu_tipo = endutih_vivhogar[(endutih_vivhogar.COMPU==True)&(endutih_vivhogar.P4_2_1==1)][["ESTRATO", "Grupo", "FAC_HOG"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_compu_tipo.columns = ["Ordenador"]
grupo_compu_tipo

Ordenador
ESTRATO Grupo           
1       11        198187
2       20       2082084
        21         36247
        22         17407
        23          8069
3       30         47775
        31         13896
        32         34766
        33       1480728
        34          8115
4       40         25260
        41          5319
        42       1029916
        43         42334
        44          5032

In [26]:
grupo_compu_tipo["Laptop"] = endutih_vivhogar[(endutih_vivhogar.COMPU==True)&(endutih_vivhogar.P4_2_2==1)][["ESTRATO", "Grupo", "FAC_HOG"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_compu_tipo["Tablet"] = endutih_vivhogar[(endutih_vivhogar.COMPU==True)&(endutih_vivhogar.P4_2_3==1)][["ESTRATO", "Grupo", "FAC_HOG"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_compu_tipo

Ordenador   Laptop   Tablet
ESTRATO Grupo                             
1       11        198187   785620   353407
2       20       2082084  4828180  2437367
        21         36247    45183    26237
        22         17407    36152    21112
        23          8069    24177    13594
3       30         47775    65313    39595
        31         13896    31700    19362
        32         34766    65720    32345
        33       1480728  3282566  1621163
        34          8115    19804    12515
4       40         25260    55985    27349
        41          5319    29013    10505
        42       1029916  2242544  1263455
        43         42334    86069    54325
        44          5032    11051     5890

In [27]:
grupo_compu_tipo_rel_grupo = grupo_compu_tipo/np.array(grupo_compu["Sí"]).repeat(3).reshape(15,3)
grupo_compu_tipo_rel_grupo

Ordenador    Laptop    Tablet
ESTRATO Grupo                               
1       11      0.178571  0.707861  0.318428
2       20      0.302422  0.701291  0.354027
        21      0.453751  0.565615  0.328443
        22      0.279307  0.580084  0.338757
        23      0.210883  0.631864  0.355278
3       30      0.491366  0.671744  0.407234
        31      0.312917  0.713835  0.436003
        32      0.424914  0.803236  0.395324
        33      0.345900  0.766812  0.378706
        34      0.306655  0.748366  0.472924
4       40      0.406266  0.900428  0.439864
        41      0.159094  0.867795  0.314211
        42      0.381039  0.829676  0.467442
        43      0.426720  0.867561  0.547587
        44      0.425683  0.934862  0.498266

Las laptops son claramente dominates a comparación de las otras opciones

## Número de integrantes Promedio*
*Mayores de 15 años

In [28]:
integrantes = endutih_res[["ESTRATO", "Grupo", "UPM", "VIV_SEL", "HOGAR", "FAC_HOGAR"]].groupby(["ESTRATO", "Grupo", "UPM", "VIV_SEL", "HOGAR"]).agg(["count"])
integrantes.columns = ["Total"]

In [29]:
endutih_vivhogar["Integrantes_total"] = np.array(integrantes["Total"])*endutih_vivhogar["FAC_HOG"]

In [30]:
grupo_integrantes = endutih_vivhogar[["ESTRATO", "Grupo", "Integrantes_total"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_integrantes = grupo_integrantes.div(dist_grupos.FAC_HOG, axis=0)
grupo_integrantes

Integrantes_total
ESTRATO Grupo                   
1       11              3.834178
2       20              3.620158
        21              3.956064
        22              3.444718
        23              3.413505
3       30              3.273240
        31              3.294421
        32              3.222666
        33              3.293917
        34              3.578411
4       40              3.104074
        41              3.456909
        42              3.106736
        43              3.149561
        44              3.339987

De forma general, parece que se tomaron 3 residentes por hogar

## Línea telefónica

In [31]:
grupo_tel = pd.pivot_table(data=endutih_vivhogar, columns="P5_5", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_tel.columns = pd.Series(["Sí", "No"], name="Línea telefónica en el hogar")
grupo_tel

Línea telefónica en el hogar       Sí        No
ESTRATO Grupo                                  
1       11                     705474   6279365
2       20                    5940495  11354706
        21                     105090      3408
        22                       3415    107462
        23                      18818     73650
3       30                     114319     23915
        31                      43488     37865
        32                     101976      2822
        33                    4158208   2948510
        34                        795     42356
4       40                      69070      9374
        41                      16703     30221
        42                    2381615    988209
        43                     106791       102
        44                        972     14858

### Relativo al grupo

In [32]:
grupo_tel_rel_grupo = grupo_tel.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_tel_rel_grupo.style.format(precision=4)

Llama la atención cómo la línea telefónica no es tan usada como el Internet

## Actividad Laboral

In [33]:
grupo_trabajo = pd.pivot_table(data=endutih_res, columns="P3_10", values= "FAC_HOGAR", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_trabajo.columns = pd.Series(["Trabajó","No fue al trabajo", "Buscó trabajo", "Está jubilado", "Se dedicó a estudiar", "Quehaceres del hogar", "Limitación para trabajar", "No trabajó"], name="Actividad laboral")
grupo_trabajo

Actividad laboral   Trabajó  No fue al trabajo  Buscó trabajo  Está jubilado  \
ESTRATO Grupo                                                                  
1       11          9525436             215513         295312         133837   
2       20         25754175             700073        1034490        1511948   
        21           185196               2063          11385          24360   
        22           182042               4515           2772           3774   
        23           118521               3454           6027           8836   
3       30           219058               7646           1556          33219   
        31           102179               3954           3452          17669   
        32           161331               2683           1247          30508   
        33         10457646             287738         431892        1412323   
        34            66138               2324           2826           9713   
4       40           111496               1551            750          11313   
        41            60574               4663           1802           7005   
        42          4907200             106583         129243         671135   
        43           163207               2423            791          21823   
        44            26705               2451              0           4676   

Actividad laboral  Se dedicó a estudiar  Quehaceres del hogar  \
ESTRATO Grupo                                                   
1       11                      2471815               6314638   
2       20                      7281888              10710202   
        21                        56621                 75116   
        22                        56983                 69991   
        23                        42554                 62293   
3       30                        60321                 65070   
        31                        27470                 39400   
        32                        46862                 60851   
        33                      2899165               3242856   
        34                        16935                 24025   
4       40                        35145                 45138   
        41                        13844                 12301   
        42                      1494711               1292718   
        43                        51777                 47672   
        44                         8277                  6962   

Actividad laboral  Limitación para trabajar  No trabajó  
ESTRATO Grupo                                            
1       11                           385096     1265210  
2       20                           813520     2743109  
        21                             3484       14111  
        22                             3299       22769  
        23                             8023       12073  
3       30                             5656       20641  
        31                             1229       14633  
        32                             4862       16870  
        33                           219067      965542  
        34                              326        9265  
4       40                             2163        5402  
        41                              663        7647  
        42                            48372      376044  
        43                              963        9020  
        44                             1360        1088

### Relativo al grupo

In [34]:
grupo_trabajo_rel_grupo = grupo_trabajo.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_trabajo_rel_grupo.style.format(precision=4)

De manera general, podemos ver que la mayoría d elos integrantes de los grupos son población económicamente activa, sin embargo, llama la atención que en el grupo 11, haya un alto porcentaje de residentes que se dedican a los quehaceres del hogar

## EDUCACIÓN

In [35]:
grupo_edu = pd.pivot_table(data=endutih_res, columns=["NIVEL"], values= "FAC_HOGAR", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_edu.columns = pd.Series(["Ninguno", "Preescolar", "Primaria", "Secundaria","Normal básica", "Estudio técnico", "Preparatoria", "Estudio técnico superior", "Licenciatura o ingeniería", "Especialidad", "Maestría", "Doctorado", "No sabe"], name="Nivel Educativo")
grupo_edu

Nivel Educativo  Ninguno  Preescolar  Primaria  Secundaria  Normal básica  \
ESTRATO Grupo                                                               
1       11       2328351      555378  10344537     6600272           8154   
2       20       2316109      928185  17869900    16862237          59624   
        21         10221        1590     85454      101620            300   
        22          6149        9369    101749      120734              0   
        23         15785        5526     82985       74972              0   
3       30          5767        6838     75262       83698           1116   
        31          6579           0     44995       48468            426   
        32          8222        4279     46359       57706           1863   
        33        354243      270736   4586229     5072811          51231   
        34          3123        1290     31860       41553              0   
4       40           962        1863     26124       34876           1038   
        41             0        1806     16167       12500              0   
        42         46441      110220   1270354     1457802          28177   
        43          2031        3960     23649       33527            145   
        44           584        2469      6945        7366              0   

Nivel Educativo  Estudio técnico  Preparatoria  Estudio técnico superior  \
ESTRATO Grupo                                                              
1       11                 51324       3072305                     73902   
2       20                569721      10637604                    732938   
        21                  4681        100593                      2737   
        22                  3876         79380                      2466   
        23                  3401         52727                       618   
3       30                 19658         83810                      7724   
        31                  1999         31191                      4737   
        32                  3624         77996                     12764   
        33                473072       4727311                    534272   
        34                   287         23861                      1063   
4       40                  2837         37458                      4512   
        41                  1674         26472                       645   
        42                232554       1862674                    281305   
        43                  5320         37381                      3349   
        44                     0         14650                       200   

Nivel Educativo  Licenciatura o ingeniería  Especialidad  Maestría  Doctorado  \
ESTRATO Grupo                                                                   
1       11                          987122          5582     22756       2323   
2       20                         6680481         49808    297437      41919   
        21                           76376             0      7217          0   
        22                           48998             0      2464        283   
        23                           42766             0      3304       1064   
3       30                          129959           800     10859       3733   
        31                           67996           397      6503          0   
        32                          108482          1477      8028        904   
        33                         5205682         51670    376964      69993   
        34                           33240           158      2066        478   
4       40                           95168          2813     12682       3291   
        41                           47026             0      9558          0   
        42                         3839510         93487    451371      95248   
        43                          168344          9257     19227       4004   
        44                           21421             0      1710          0   

Nivel Educativo  No

### Relativo al grupo

In [36]:
grupo_edu_rel_grupo = grupo_edu.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_edu_rel_grupo.style.format(precision=4)

## Celular

In [37]:
grupo_cel = pd.pivot_table(data=endutih_vivhogar, columns="P4_1_6", values= "FAC_HOG", index=["ESTRATO", "Grupo"], aggfunc='sum', fill_value=0)
grupo_cel.columns = pd.Series(["Sí", "No"], name="Celular en el hogar")
grupo_cel

Celular en el hogar        Sí       No
ESTRATO Grupo                         
1       11            5672484  1312355
2       20           16103426  1191775
        21             103537     4961
        22             108007     2870
        23              83789     8679
3       30             133743     4491
        31              67281    14072
        32             103209     1589
        33            6798418   308300
        34              42420      731
4       40              76751     1693
        41              42886     4038
        42            3312070    57754
        43             106053      840
        44              15558      272

### Relativo al grupo

In [38]:
grupo_cel_rel_grupo = grupo_cel.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_cel_rel_grupo.style.format(precision=4)

### Detalles

In [39]:
endutih_vivhogar["Movil"] = endutih_vivhogar["FAC_HOG"]*endutih_vivhogar["P4_1_6_1"]
endutih_vivhogar["Smartphone"] = endutih_vivhogar["FAC_HOG"]*endutih_vivhogar["P4_1_6_2"]

In [40]:
grupo_cel_tipo = endutih_vivhogar[(endutih_vivhogar.P4_1_6==1)][["ESTRATO", "Grupo", "FAC_HOG",]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_cel_tipo.columns = ["Total"]
grupo_cel_tipo["Móvil"]= endutih_vivhogar[(endutih_vivhogar.P4_1_6==1)&(endutih_vivhogar.P4_1_6_1>0)][["ESTRATO", "Grupo", "Movil",]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_cel_tipo["Smartphone"] = endutih_vivhogar[(endutih_vivhogar.P4_1_6==1)&(endutih_vivhogar.P4_1_6_2>0)][["ESTRATO", "Grupo", "Smartphone"]].groupby(["ESTRATO", "Grupo"]).agg("sum")
grupo_cel_tipo

Total       Móvil  Smartphone
ESTRATO Grupo                                  
1       11      5672484  11641015.0   9943009.0
2       20     16103426  40562730.0  36991337.0
        21       103537    314914.0    307501.0
        22       108007    295927.0    311581.0
        23        83789    208323.0    199622.0
3       30       133743    414680.0    387996.0
        31        67281    166236.0    169948.0
        32       103209    298661.0    277190.0
        33      6798418  18108388.0  17183848.0
        34        42420    123270.0    117358.0
4       40        76751    205059.0    199610.0
        41        42886    116878.0    113546.0
        42      3312070   8847228.0   8629132.0
        43       106053    304052.0    295394.0
        44        15558     42980.0     42780.0

In [41]:
grupo_cel_tipo_promedio = grupo_cel_tipo[["Móvil", "Smartphone"]].div(grupo_cel_tipo.Total, axis=0)
grupo_cel_tipo_promedio

Móvil  Smartphone
ESTRATO Grupo                      
1       11     2.052190    1.752849
2       20     2.518888    2.297110
        21     3.041560    2.969962
        22     2.739887    2.884822
        23     2.486281    2.382437
3       30     3.100573    2.901057
        31     2.470772    2.525943
        32     2.893750    2.685715
        33     2.663618    2.527625
        34     2.905941    2.766572
4       40     2.671744    2.600748
        41     2.725318    2.647624
        42     2.671208    2.605359
        43     2.866982    2.785343
        44     2.762566    2.749711

## Sexo

In [42]:
grupo_sexo = pd.pivot_table(data=endutih_res, index=["ESTRATO","Grupo"], columns="SEXO", values="FAC_HOGAR", aggfunc="sum")
grupo_sexo.columns = pd.Series(["Hombre","Mujer"], name="SEXO")
grupo_sexo

SEXO             Hombre     Mujer
ESTRATO Grupo                    
1       11     13202300  13864855
2       20     30487305  32364627
        21       198773    222898
        22       193702    231715
        23       149508    163263
3       30       206177    254475
        31        96885    131775
        32       159500    200386
        33     11362394  12025241
        34        76740     79702
4       40       108811    126157
        41        54656     71747
        42      4932464   5465360
        43       154345    169817
        44        25219     35530

### Relativo al grupo

In [43]:
grupo_sexo_rel_grupo = grupo_sexo.apply(lambda x: x.div(x.sum()), axis=1)*100
grupo_sexo_rel_grupo

SEXO              Hombre      Mujer
ESTRATO Grupo                      
1       11     48.776090  51.223910
2       20     48.506552  51.493448
        21     47.139357  52.860643
        22     45.532266  54.467734
        23     47.801107  52.198893
3       30     44.757648  55.242352
        31     42.370769  57.629231
        32     44.319590  55.680410
        33     48.582911  51.417089
        34     49.053323  50.946677
4       40     46.308859  53.691141
        41     43.239480  56.760520
        42     47.437464  52.562536
        43     47.613539  52.386461
        44     41.513441  58.486559

## IDTMex

In [9]:
IDTMex_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO", "Grupo"], aggfunc=IDTMex.IDTMex, fill_value=0)
IDTMex_grupos_edo.columns = estados.ENT
IDTMex_grupos_edo

C:\Users\jazmi\AppData\Local\Temp\ipykernel_18300\3104121406.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  IDTMex_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO", "Grupo"], aggfunc=IDTMex.IDTMex, fill_value=0)


ENT                Ags.      B.C.     B.C.S     Camp.     Coah.       Col.  \
ESTRATO Grupo                                                                
1       11     0.000000  3.509422  4.322510  3.152224  2.249533   3.963927   
2       20     4.715311  5.865413  5.453550  5.451482  4.782666   5.155523   
        21     7.212964  8.055016  8.173970  8.091542  6.232668   7.447854   
        22     6.318044  6.640156  6.356524  5.950369  5.743930   6.035900   
        23     4.545903  4.770804  5.154370  4.265302  4.278083   5.150232   
3       30     7.340089  7.944129  7.368104  7.188754  6.801749   7.772515   
        31     5.516981  7.971836  6.597414  6.250826  4.375062   6.547206   
        32     7.064853  8.008826  5.000000  7.897286  8.444444   8.324895   
        33     6.177313  7.108091  6.580328  6.732477  6.197529   6.695836   
        34     6.090581  8.034558  6.377292  7.475884  7.510594   5.652965   
4       40     6.939488  8.812334  8.475832  5.957107  8.075184   8.754785   
        41     7.994415  9.466667  6.858870  8.666667  9.666667   7.674468   
        42     7.388430  7.831090  7.710550  7.000035  7.836227   7.442446   
        43     8.464286  8.916199  7.666667  7.333333  8.008005  10.000000   
        44     6.579902  7.751601  7.666667  6.166667  0.000000   8.566833   

ENT               Chis.     Chih.    C.D.Mx      Dgo.      Gto.      Gro.  \
ESTRATO Grupo                                                               
1       11     2.136097  2.704921  4.913072  3.019904  3.679311  3.401621   
2       20     4.516427  5.196660  6.676073  4.547788  4.341657  5.353832   
        21     6.791951  0.000000  7.723988  7.666667  7.163437  6.664675   
        22     6.536865  7.833333  6.546627  5.737500  6.787817  6.436400   
        23     3.061872  0.000000  4.666667  2.888889  3.965488  4.573409   
3       30     6.333333  6.371514  8.303545  0.000000  7.598358  1.666667   
        31     6.963058  5.290554  7.253774  6.461579  6.536398  5.698692   
        32     0.000000  8.198852  8.796732  0.000000  8.150335  0.000000   
        33     6.618886  6.295855  7.448001  6.276785  6.313263  7.533533   
        34     0.000000  9.000000  8.591047  5.833333  6.578192  0.000000   
4       40     0.000000  9.000000  9.154944  7.809745  8.356411  0.000000   
        41     7.666667  9.000000  6.904792  5.390363  7.666667  0.000000   
        42     6.848023  7.667511  8.488164  7.075716  7.688000  8.416667   
        43     8.666667  4.000000  9.404851  0.000000  9.086759  0.000000   
        44     0.000000  0.000000  7.888889  0.000000  6.818362  0.000000   

ENT                Hgo.      Jal.  Edo. Méx.     Mich.      Mor.       Nay.  \
ESTRATO Grupo                                                                 
1       11     2.955906  2.926440   3.197817  2.357045  4.041712   2.402178   
2       20     4.667045  4.989033   5.654172  4.491235  5.307903   4.775072   
        21     6.520644  7.873503   7.621675  7.308558  7.332781   8.003444   
        22     4.786023  6.831785   6.162905  5.713394  5.696650   6.529955   
        23     5.007581  5.635662   4.666667  4.397901  3.773863   3.631431   
3       30     0.000000  7.181075   7.299473  7.746778  6.993305   8.871010   
        31     1.666667  3.916692   4.486712  6.556072  6.016331   5.599994   
        32     7.127682  7.438901   7.932486  6.969107  8.165602   8.222222   
        33     6.425261  6.164887   6.861031  5.999945  6.379561   6.985303   
        34     0.000000  5.983468   0.000000  5.749542  6.522020   6.406769   
4       40     0.000000  9.590595   7.092048  7.974406  0.000000   8.138770   
        41     0.000000  7.935612   0.000000  5.417016  7.345476   0.000000   
        42     7.234888  7.905820   7.828485  7.390869  7.408084   8.047967   
        43     0.000000  8.970157   8.671252  0.000000  8.507569   8.666667   
        44     0.000000  0.000000   0.000000  0.000000  5.666667  10.000000   

ENT                

In [10]:
IDTMex_grupos_edo.to_csv(base_path+"IDTMex_grupos_edo.csv")

In [13]:
IDTMex_dominio_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","DOMINIO","Grupo"], aggfunc=IDTMex.IDTMex, fill_value=0)
IDTMex_dominio_grupos_edo.columns = estados.ENT
IDTMex_dominio_grupos_edo

C:\Users\jazmi\AppData\Local\Temp\ipykernel_18300\1645834172.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  IDTMex_dominio_grupos_edo = pd.pivot_table(data=endutih_completa, columns="ENT", index=["ESTRATO","DOMINIO","Grupo"], aggfunc=IDTMex.IDTMex, fill_value=0)


ENT                        Ags.      B.C.     B.C.S     Camp.     Coah.  \
ESTRATO DOMINIO Grupo                                                     
1       R       11     0.000000  3.411529  3.066407  3.058415  2.214322   
        U       11     0.000000  5.046326  5.006640  3.352829  2.779101   
2       R       20     4.703610  4.703621  5.115594  0.000000  3.136870   
                21     6.985203  0.000000  8.111111  0.000000  0.000000   
                22     6.028424  7.555556  4.996379  0.000000  0.000000   
                23     5.333333  0.000000  6.423571  0.000000  1.666667   
        U       20     4.724455  6.039523  5.549181  5.451482  5.069195   
                21     7.310672  8.055016  8.180849  8.091542  6.232668   
                22     6.390171  6.379414  7.138617  5.950369  5.743930   
                23     4.395722  4.770804  3.206079  4.265302  5.030954   
3       U       30     7.340089  7.944129  7.368104  7.188754  6.801749   
                31     5.516981  7.971836  6.597414  6.250826  4.375062   
                32     7.064853  8.008826  5.000000  7.897286  8.444444   
                33     6.177313  7.108091  6.580328  6.732477  6.197529   
                34     6.090581  8.034558  6.377292  7.475884  7.510594   
4       U       40     6.939488  8.812334  8.475832  5.957107  8.075184   
                41     7.994415  9.466667  6.858870  8.666667  9.666667   
                42     7.388430  7.831090  7.710550  7.000035  7.836227   
                43     8.464286  8.916199  7.666667  7.333333  8.008005   
                44     6.579902  7.751601  7.666667  6.166667  0.000000   

ENT                         Col.     Chis.     Chih.    C.D.Mx      Dgo.  \
ESTRATO DOMINIO Grupo                                                      
1       R       11      3.963927  1.927872  2.596100  0.000000  2.493391   
        U       11      0.000000  2.854510  6.896296  4.913072  7.303790   
2       R       20      4.474019  3.168165  4.208359  6.175713  3.847917   
                21      0.000000  0.000000  0.000000  0.000000  0.000000   
                22      0.000000  0.000000  7.833333  0.000000  6.888889   
                23      7.333333  0.000000  0.000000  0.000000  2.888889   
        U       20      5.267386  4.673123  5.418688  6.683631  4.845320   
                21      7.447854  6.791951  0.000000  7.723988  7.666667   
                22      6.035900  6.536865  0.000000  6.546627  2.666667   
                23      4.915124  3.061872  0.000000  4.666667  0.000000   
3       U       30      7.772515  6.333333  6.371514  8.303545  0.000000   
                31      6.547206  6.963058  5.290554  7.253774  6.461579   
                32      8.324895  0.000000  8.198852  8.796732  0.000000   
                33      6.695836  6.618886  6.295855  7.448001  6.276785   
                34      5.652965  0.000000  9.000000  8.591047  5.833333   
4       U       40      8.754785  0.000000  9.000000  9.154944  7.809745   
                41      7.674468  7.666667  9.000000  6.904792  5.390363   
                42      7.442446  6.848023  7.667511  8.488164  7.075716   
                43     10.000000  8.666667  4.000000  9.404851  0.000000   
                44      8.566833  0.000000  0.000000  7.888889  0.000000   

ENT                        Gto.      Gro.      Hgo.      Jal.  Edo. Méx.  \
ESTRATO DOMINIO Grupo                                                      
1       R       11     3.622589  3.012724  2.940582  2.942555   2.878523   
        U       11     4.612415  4.227316  4.626263  2.822222   3.989717   
2       R       20     3.528747  0.000000  4.145539  4.509161   4.843588   
                21     0.000000  0.000000  6.867514  0.000000   8.111111   
                22     7.444444  0.000000  5.025865  0.000000   0.000000   
                23     4.768689  0.000000  0.000000  5.666667   0.000000   
        U       20     4.660503  5.353832  4.906963  5.146406   5.74

In [14]:
IDTMex_dominio_grupos_edo.to_csv(base_path+"IDTMex_dominio_grupos_edo.csv")

# Estrato Bajo

## Grupo 11

In [61]:
grupo11_vivhogar = endutih_vivhogar.loc[endutih_vivhogar.Grupo==11]
grupo11_res = endutih_res.loc[endutih_res.Grupo==11]
grupo11_usr = endutih_usr.loc[endutih_usr.Grupo==11]
grupo11_usr2 = endutih_usr2.loc[endutih_usr2.Grupo==11]

### Actividad laboral por sexo

In [62]:
sit_trabajo = ["Trabajó","No fue al trabajo", "Buscó trabajo", "Está jubilado", "Se dedicó a estudiar", "Quehaceres del hogar", "Limitación para trabajar", "No trabajó"]
sexos = ["Hombre", "Mujer"]
combinaciones = [(situacion,sexo) for situacion in sit_trabajo for sexo in sexos]

In [63]:
trabajo_sexo_grupo11 = pd.pivot_table(data=grupo11_res, columns="SEXO", index="P3_10", values="FAC_HOGAR", aggfunc="sum")
trabajo_sexo_grupo11.index = pd.Series(sit_trabajo, name="Actividad laboral")
trabajo_sexo_grupo11.columns = pd.Series(sexos, name="SEXO")
trabajo_sexo_grupo11

SEXO,Hombre,Mujer
Actividad laboral,,
Trabajó,7038453,2486983
No fue al trabajo,156245,59268
Buscó trabajo,252706,42606
Está jubilado,100942,32895
Se dedicó a estudiar,1118822,1352993
Quehaceres del hogar,123600,6191038
Limitación para trabajar,231152,153944
No trabajó,875157,390053


In [64]:
trabajo_sexo_grupo11_rel_grupo = trabajo_sexo_grupo11.apply(lambda x: x.div(x.sum()), axis=1)*100
trabajo_sexo_grupo11_rel_grupo

SEXO,Hombre,Mujer
Actividad laboral,,
Trabajó,73.891137,26.108863
No fue al trabajo,72.499107,27.500893
Buscó trabajo,85.572547,14.427453
Está jubilado,75.421595,24.578405
Se dedicó a estudiar,45.263177,54.736823
Quehaceres del hogar,1.957357,98.042643
Limitación para trabajar,60.024513,39.975487
No trabajó,69.170889,30.829111


In [65]:
trabajo_sexo_grupo11_rel_sexo = trabajo_sexo_grupo11.apply(lambda x: x.div(x.sum()), axis=0)*100
trabajo_sexo_grupo11_rel_sexo

SEXO,Hombre,Mujer
Actividad laboral,,
Trabajó,71.116482,23.221607
No fue al trabajo,1.578698,0.553401
Buscó trabajo,2.553340,0.397823
Está jubilado,1.019917,0.307149
Se dedicó a estudiar,11.304570,12.633247
Quehaceres del hogar,1.248854,57.807331
Limitación para trabajar,2.335558,1.437415
No trabajó,8.842580,3.642026
